In [22]:
# 외부 IP 확인하기

import requests

print(requests.get("http://ip.jsontest.com").json()['ip'])

35.227.157.253


In [23]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### DB 적재

In [ ]:
# DB 연결
from pymongo import MongoClient

client = MongoClient(host='3.37.202.252', port=27017)

In [ ]:
print(client.list_database_names())

['admin', 'config', 'local', 'youtube_comment_database']


In [ ]:
# Collection 접근
db = client['youtube_comment_database']
collection = db['youtube_comment_collection']

In [ ]:
import json
a_json = open('/content/drive/MyDrive/add_comments_data.json', encoding='utf-8')
a_dict = json.load(a_json)

In [ ]:
collection.insert_one(document=a_dict)

---
### Modeling

In [24]:
import pandas as pd
import urllib.request
import json
import numpy as np
import re
import os
import sys
import collections
import tensorflow as tf 

from keras import *
from keras.layers import *
from tensorflow.keras.utils import to_categorical
from tensorflow import keras
from bpemb import BPEmb

In [25]:
urllib.request.urlretrieve("https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt", filename="ratings_train.txt")
urllib.request.urlretrieve("https://raw.githubusercontent.com/e9t/nsmc/master/ratings_test.txt", filename="ratings_test.txt")

('ratings_test.txt', <http.client.HTTPMessage at 0x7f8fc305fc90>)

In [26]:
train_data = pd.read_csv('ratings_train.txt', sep='\t').dropna()
test_data = pd.read_csv('ratings_test.txt', sep='\t').dropna()

# column 이름 변경 
train_data.rename(columns = {'document' : 'review'}, inplace = True)
test_data.rename(columns = {'document' : 'review'}, inplace = True)

# csv 파일로 저장
train_data.to_csv('train_data.csv', index=False)
test_data.to_csv('test_data.csv', index=False)

In [27]:
d_emb = 300
vocab_size = 200000
batch_size = 128
seq_max = 200

In [28]:
TRAIN_DIR = './ratings_train.txt'
TEST_DIR = './ratings_test.txt'

In [29]:
# 참이면 사전 훈련된 벡터로 가중치 포함 초기화 
pre_trained = True

# True이면 4 CNN 레이어 사용
quad_layer = True

filter_size = 100

if quad_layer:
  windows = [2, 3, 4, 5]
else: 
  windows = [2, 3, 4]

In [30]:
if pre_trained:
    bpemb_ko = BPEmb(lang="ko", dim=d_emb, vs=vocab_size) 
else:
    bpemb_ko = BPEmb(lang="ko", vs=vocab_size)

downloading https://nlp.h-its.org/bpemb/ko/ko.wiki.bpe.vs200000.model


100%|██████████| 4234933/4234933 [00:01<00:00, 3702894.57B/s]


downloading https://nlp.h-its.org/bpemb/ko/ko.wiki.bpe.vs200000.d300.w2v.bin.tar.gz


100%|██████████| 224212701/224212701 [00:13<00:00, 16724103.27B/s]


In [31]:
def parsing(data):
    first_t = data.find('\t')
    second_t = data[first_t + 1:].find('\t') + first_t + 1
    _id = data[:first_t]
    document = data[first_t + 1:second_t]
    label = data[second_t + 1:]
    
    return _id, document, label

In [32]:
def add_padding(arr, max_len):
    results = []
    for tmp in arr:
        for i in range(len(tmp), max_len):
            tmp.append(0)
        results.append(tmp)

    return results

In [33]:
def raw_to_data(raw):
    x = []
    y = []
    for data in raw[1:]:

        _, document, label = parsing(data)

        tmp = bpemb_ko.encode_ids(document)

        if seq_max < len(tmp):
            continue

        if len(tmp) == 0:
            continue
    
        x.append(tmp)
        y.append(label)
        
    return np.array(add_padding(x.copy(), seq_max)), to_categorical(np.array(y.copy(), dtype='int32'))

In [34]:
with open(TRAIN_DIR, "r", encoding='utf8') as f:
    train_raw_data = f.readlines()

In [35]:
with open(TEST_DIR, "r", encoding='utf8') as f:
    test_raw_data = f.readlines()

In [36]:
x_train, y_train = raw_to_data(train_raw_data)
x_test, y_test = raw_to_data(test_raw_data)

print('x_train shape: ', x_train.shape)
print('y_train shape: ', y_train.shape)
print('x_test shape: ', x_test.shape)
print('y_test shape: ', y_test.shape)

x_train shape:  (149995, 200)
y_train shape:  (149995, 2)
x_test shape:  (49997, 200)
y_test shape:  (49997, 2)


In [37]:
class SentimentalCNN:
    def __init__(self, vocab_size, d_emb, seq_max, weight=None, trainable=False):

        if weight is None:
            self.emb_layer = Embedding(vocab_size, d_emb, input_length=seq_max)
            
        else:
            self.emb_layer = Embedding(vocab_size, d_emb, input_length=seq_max, weights=[weight], trainable=trainable)

            self.reshape_layer = Reshape((seq_max, d_emb, 1), input_shape=(seq_max, d_emb)) 
            
            self.bi_cnn = Conv2D(filter_size * 1, kernel_size=(windows[0], d_emb), activation='relu', input_shape=(seq_max, d_emb, 1))
            self.tri_cnn = Conv2D(filter_size * 1, kernel_size=(windows[1], d_emb), activation='relu', input_shape=(seq_max, d_emb, 1))
            self.quad_cnn = Conv2D(filter_size * 1, kernel_size=(windows[2], d_emb), activation='relu', input_shape=(seq_max, d_emb, 1))
            
            if quad_layer:
                self.penta_cnn = Conv2D(filter_size * 1, kernel_size=(windows[3], d_emb), activation='relu', input_shape=(seq_max, d_emb, 1))
                
            self.output_layer = Dense(2, activation='softmax')

    def compile(self, optimizer="adam"):
        _input = Input(shape=(None, ), dtype='int32')

        emb = self.emb_layer(_input)
        reshape_emb = self.reshape_layer(emb)
            
        bi_res = self.bi_cnn(reshape_emb)
        tri_res = self.tri_cnn(reshape_emb)
        quad_res = self.quad_cnn(reshape_emb)
            
        bi_res = Dropout(0.3)(bi_res)
        tri_res = Dropout(0.3)(tri_res)
        quad_res = Dropout(0.3)(quad_res)
                
        a = MaxPool2D(pool_size=(seq_max - windows[0] - 1, 1))(bi_res)
        b = MaxPool2D(pool_size=(seq_max - windows[1] - 1, 1))(tri_res)
        c = MaxPool2D(pool_size=(seq_max - windows[2] - 1, 1))(quad_res)
            
        if quad_layer:
            penta_res = self.penta_cnn(reshape_emb)
            penta_res = Dropout(0.3)(penta_res)
            d = MaxPool2D(pool_size=(seq_max - windows[3] - 1, 1))(penta_res)
            result = Concatenate(axis=-1)([a, b, c, d])
            
        else:
            result = Concatenate(axis=-1)([a, b, c])
                
        result = Flatten()(result)
        result = self.output_layer(result)

        self.model = Model(_input, result)
        self.model.compile(
            loss='categorical_crossentropy',
            optimizer=optimizer,
            metrics=['accuracy']
        )
            
    def predict(self, _input):
        x = bpemb_ko.encode_ids(_input)
        x = np.array(add_padding([x], seq_max))
                    
        res = self.model.predict(x)
                
        if res[0][0] < res[0][1]:
            print(_input + "\n: 긍정")
                    
        else:
            print(_input + "\n: 부정")

In [38]:
if pre_trained:
    sent_cnn = SentimentalCNN(
        vocab_size=vocab_size,
        d_emb=d_emb,
        seq_max=seq_max,
        weight=bpemb_ko.vectors,
        trainable=True
    )
            
else:
    sent_cnn = SentimentalCNN(
        vocab_size=vocab_size,
        d_emb=d_emb,
        seq_max=seq_max
    )

    
sent_cnn.compile(tf.optimizers.Adam(lr=0.001, decay=1e-6))
sent_cnn.model.build(input_shape=(seq_max, ))
sent_cnn.model.summary()
hist = sent_cnn.model.fit(x_train, y_train,
                          epochs=3,
                          batch_size=batch_size,
                          verbose=1,
                          validation_data=(x_test, y_test))

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, None)]       0           []                               
                                                                                                  
 embedding (Embedding)          (None, None, 300)    60000000    ['input_1[0][0]']                
                                                                                                  
 reshape (Reshape)              (None, 200, 300, 1)  0           ['embedding[0][0]']              
                                                                                                  
 conv2d (Conv2D)                (None, 199, 1, 100)  60100       ['reshape[0][0]']                
                                                                                              

---
### 감정 분석

In [12]:
post_id_1 = str('62c3d51d0150cc08b72e4368') # video_comments
post_id_2 = str('62c3c0441138d063a9b71e3b') # video_info

In [13]:
from bson.objectid import ObjectId
video_comments_bson_id = ObjectId(post_id_1)
video_comments = collection.find_one({"_id": video_comments_bson_id})
video_info_bson_id = ObjectId(post_id_2)
video_info = collection.find_one({"_id": video_info_bson_id})

In [14]:
del video_comments['_id']
del video_info['_id']

In [15]:
## video_comments 재구성
for movie_name in  video_info:
    video_comments[movie_name] = video_comments[movie_name]['Comments']

In [16]:
import pandas as pd

def pad_dict_list(dict_list, padel=''):
    lmax = 0
    for lname in dict_list.keys():
        lmax = max(lmax, len(dict_list[lname]))
    for lname in dict_list.keys():
        ll = len(dict_list[lname])
        if  ll < lmax:
            dict_list[lname] += [padel] * (lmax - ll)
    return dict_list

video_comments = pad_dict_list(video_comments)
df = pd.DataFrame(video_comments)

In [39]:
reviews = df['명량 '].to_list()

for sen in reviews:
  result = sent_cnn.predict(sen)

이순신장군님 감사합니다
: 긍정
한산대첩에서 당한 기억 때문에 대장선 하나만 나와있는 것도 무슨 작전인 줄 아는 거 디테일하네
: 부정
포에 미친 민족
: 부정
난중일기는 꼭 읽어보시고 다시한번 충무공께 감사하시기 바랍니다 우린 충무공이 아니였다면 혼다나 나까지마가 되었을 것입니다
: 긍정
이거 보고 국뽕에 취할려고 하는데 추천영화가 악마를 보았다로 연결되네
: 긍정
그 누구도 이순신 장군님께 모욕댓글 쓰지마라 늬들이 그시대 살았어 국가에 충성이란 무슨 의미를 아나 일본 압잡이 도맡아서 할놈들 좋은글 에 꼭 이상한넘들 이씀 그넘들은 미친넘 병쉰같은 넘들이지 아순신장군님덕에 이시대 내가 있는거임
: 부정
못보겠는거 억지 백병전 까마귀절규 나룻배 레카
: 부정
참 명 감독에 명배우들의 열연이었습니다 최민식이란 배우의 연기력은 많은 대사가 없이도 정말 잘 소화해 내는 진짜 명배우네요
: 긍정
필요없는 신파장면 이순신을 빼면 나머지 인물들이 병풍인 영화 다만 최민식의 연기가 빛남
: 부정
오와우
: 긍정
스님들 쎄다
: 부정
개쌉노잼
: 부정
스님이 왜 전투를 해요
: 긍정
명량이 이래서 욕먹었구나 영화자체는 재미있지만 실제 명량대첩은 이순신성웅이 탑승한 대장선 한척으로 쓸어버렸고 공식적인 사망자는 명임 그냥 능욕 그 자체였음 롤로 치자면 페이커 초급봇 이정도 차이
: 긍정
이순신장군이 살아 돌아오셔서 주옥순 같은 토왜들이 한국에 바글바글 한 거 보면 뭐라 하실까 진짜 모조리 칼로 베어버리겠지
: 부정
실화라는게 어이가 없음 고구려 전투 스파르타 테르모필레협곡전투 이순신 해상전 진짜 어이가 없음 말도 안되는 스토리라
: 부정
아시박 눈물나
: 긍정
개 같은 전라도 내로남불 정권이 국민 뒷통수 치려고 만든 영화
: 부정
두번 다시는 나라를 빼기는일은 없을거다 정신차려라
: 부정
물론 고증이 아쉬운 건 사실이지만 고증만으로 영화를 볼 거면 그 사람들은 영화관이 아니라 집에서 다큐를 보는 게 맞다고 생각한다 내용면에서 비판하면 몰라도 고증만으로 물고 늘어지는 사람들은 학창

In [40]:
reviews = df.iloc[0].to_list()

for sen in reviews:
  sent_cnn.predict(sen)

이순신장군님 감사합니다
: 긍정
부에서 밝혀지는 진선규의 비밀은 극한직업 부
: 긍정
모르겟다 난 갠적으로 주지훈도 눈물연기가 내느낌으론 진짜 찐으로 울컥해서 눈물이 난거같이서 진짜 슬픈장면이엇다 생각함
: 긍정
울었으면 웃으러 가자
: 긍정
마블팬들에게 가슴을 울리게한 장면
: 긍정
오늘 소개한 영화 아바타는 국내 관객 만을 기록하기도 했으며 전세계 박스오피스 위를 년간 지켰던 명작인데요 어벤져스 엔드게임 개봉 당시 잠시 위 자리를 빼앗기기도 했었지만 최근 코로나로 인한 재개봉으로 인해 엔드게임을 제치고 다시 위에 올라 섰다고 합니다 하지만 차이가 크지 않기 때문에 엔드게임도 재개봉을 하게 된다면 다시 위 자리를 탈환할 수도 있다고하네요 오늘도 시청해주셔서 감사드리고 행복한 하루 보내세요
: 긍정
둘다 글러브 꼈으면 황정민이 이겼다
: 부정
봉준호 감독님은 천재인듯
: 긍정
도둑들 은 왓챠에서 감상 가능합니다 전지현 특별전 다음 전지현 특별전 을 장식할 영화는 어떤 영화일까요 기대해주세요 감사합니다
: 긍정
이거한번더보고싶어서찾아보니있어서 한번더 눈물 콧물 쏟고갑니다 부녀간의사랑이 많이느껴져서
: 긍정
썸네일 전지현 미친듯이 예쁘다
: 긍정
근데 지니가 자유가되게해달랫지 사람이되게해달라고는안했는뎅
: 부정
여캠들 수위가 진짜 고 티 가 지리긴하지
: 부정
왕의 남자 광대 노릇 하지 마라
: 부정
원활한 이해를 위해서는 첫번째 이야기를 보고 오시는게 좋아요 첫번째 이야기
: 긍정
마지막에 아저씨 아내도 있도 아이도 있는데
: 긍정
여러분 북한은 주적입니다
: 부정
진짜 부산행 처음 봤을때 공유 무서웠는데 공유 그 장면 보고 울었는데
: 긍정
줄 착각해서 이민기 앞에 매달린 사람이 떨어졌어야
: 부정
윤석열 한동훈 보고 있나
: 부정
나간다 엄호해봐
: 부정
화질 개구리네
: 부정
이 영상의 모든 수익은 원작자에게 돌아갑니다
: 긍정
진짜 시바 존나 재미있으
: 긍정


---